# QR Decomposition

Suppose we have $A \in \mathbb{R}^{m \times n}$ representing a set of linearly
independent basis vectors (so $m \ge n$), and we want to find a series of
orthonormal vectors $q_1, q_2, \dots$ that span the successive subspaces
$\operatorname{span}(a_1)$, $\operatorname{span}(a_1, a_2)$, etc. In other words,
we want to find vectors $q_j$ and coefficients $r_{ij}$ such that
$$
\left(
\begin{array}{cccc}
| & | &        & | \\
a_1 & a_2 & \cdots & a_n \\
| & | &        & |
\end{array}
\right)
=
\left(
\begin{array}{cccc}
| & | &        & | \\
q_1 & q_2 & \cdots & q_n \\
| & | &        & |
\end{array}
\right)
\left(
\begin{array}{cccc}
r_{11} & r_{12} & \cdots & r_{1n} \\
0      & r_{22} & \cdots & r_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0      & 0      & \cdots & r_{nn}
\end{array}
\right)
$$

We can write this column-wise as
$$
a_1 = r_{11} q_1 
$$
$$
a_2 = r_{12} q_1 + r_{22} q_2
$$
$$
\vdots
$$
$$
a_n = r_{1n} q_1 + \cdots + r_{nn} q_n 
$$
so we see $q_1$ spans the space of $a_1$, and $q_1$ and $q_2$ span the space
of $\{a_1, a_2\}$, etc.

In matrix notation, we have
$$
A = \hat{Q}\,\hat{R}
$$
where $\hat{Q}$ is $m \times n$ with orthonormal columns and $\hat{R}$ is
$n \times n$ and upper triangular. This is called a \emph{reduced QR} or
\emph{economy-sized QR} factorization of $A$.

A \emph{full} QR factorization appends an additional $m-n$ orthonormal columns
to $\hat{Q}$ so it becomes a square, orthogonal matrix $Q$, which satisfies
$QQ^\top = Q^\top Q = I$. Also, we append rows of zeros to $\hat{R}$ so it
becomes an $m \times n$ matrix that is still upper triangular, called $R$. The zero entries in $R$ ``kill off'' the new columns in $Q$,
so the result is the same as $\hat{Q}\,\hat{R}$.

QR decomposition is commonly used to solve systems of linear equations.


# Gram-Schmidt Process

Algorithm: QR decomposition via Gram–Schmidt

Input:
    A ∈ R^{m×n}

Output:
    Q ∈ R^{m×n}  with orthonormal columns
    R ∈ R^{n×n}  upper triangular
    such that A = Q R

1. Let (m, n) = shape(A)
2. Initialize:
       Q = zero matrix of size m×n
       R = zero matrix of size n×n

3. For k = 0, 1, ..., n−1:          # process column k of A

       a_k = column k of A
       v   = a_k                     # working copy

       # Remove components along previously computed q_j
       For j = 0, 1, ..., k−1:

           r_{j,k} = (q_j)^T a_k     # inner product / projection coefficient
           R[j, k] = r_{j,k}

           v = v − r_{j,k} q_j       # subtract projection onto q_j

       # Now v is orthogonal to all q_0, ..., q_{k−1}

       r_{k,k} = ||v||_2             # Euclidean norm of v
       R[k, k] = r_{k,k}

       q_k = v / r_{k,k}             # normalize to unit length
       set column k of Q to q_k

4. Return Q, R


In [20]:
def qr_gram_schmidt(A):
    """
    QR decomposition of A using (classical-ish) Gram-Schmidt.
    A: m x n matrix (numpy array)
    Returns Q (m x n), R (n x n) such that A = Q @ R
    """
    A = np.array(A, dtype=float)
    m, n = A.shape

    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for k in range(n):
        v = A[:, k].copy()
        for j in range(k):
            R[j, k] = np.dot(Q[:, j], A[:, k])
            v = v - R[j, k] * Q[:, j]
        R[k, k] = np.linalg.norm(v)
        Q[:, k] = v / R[k, k]

    return Q, R

# Backward Substitution

Algorithm: Backward substitution for upper-triangular systems

Goal:
    Solve U x = b for x,
    where U ∈ R^{n×n} is upper triangular
    (i.e. U[i,j] = 0 for all i > j).

Input:
    U ∈ R^{n×n}  (upper-triangular, nonzero diagonal)
    b ∈ R^{n}    (right-hand side vector)

Output:
    x ∈ R^{n}    (solution to U x = b)

Steps:

1. Let n = number of rows of U

2. Initialize x as a length-n zero vector

3. For i = n−1, n−2, ..., 0:       # loop backwards over rows

       # Compute the sum of known terms U[i,j] * x[j]
       s = 0
       For j = i+1, i+2, ..., n−1:
           s = s + U[i, j] * x[j]

       # Solve for x[i] using the i-th equation:
       #   U[i,i] * x[i] + sum_{j=i+1}^{n-1} U[i,j] x[j] = b[i]
       # ⇒ x[i] = (b[i] − s) / U[i,i]
       x[i] = (b[i] − s) / U[i, i]

4. Return x


In [26]:
def back_substitution(U, b):
    """
    Solve U x = b for x, where U is an upper-triangular (n x n) matrix.
    Uses plain backward substitution.
    """
    U = np.asarray(U, dtype=float)
    b = np.asarray(b, dtype=float)

    n, m = U.shape
    if n != m:
        raise ValueError("U must be square")
    if b.shape[0] != n:
        raise ValueError("Dimension mismatch between U and b")

    x = np.zeros_like(b, dtype=float)

    # Go from last row up to first
    for i in range(n - 1, -1, -1):
        if U[i, i] == 0:
            raise ValueError("Matrix is singular (zero on diagonal)")

        # sum_{j=i+1}^n u_{ij} x_j
        s = 0.0
        for j in range(i + 1, n):
            s += U[i, j] * x[j]

        x[i] = (b[i] - s) / U[i, i]

    return x

# OLS via Gram-Schmidt and Backward Substitution

$$RSS(w)= \frac{1}{2} \sum_{n=1}^{N}(y_{n}-w^{\top}x_{n})^2=\frac{1}{2}||Xw-y||_{2}^{2}=\frac{1}{2}(Xw-y)^{\top}(Xw-y)=\frac{1}{2}X^{\top}Xw^2-X^{\top}yw+\frac{1}{2}y^{\top}y$$

$$\nabla RSS(w)=X^{\top}Xw-X^{\top}y \overset{!}=0$$

$$X^{\top}Xw=X^{\top}y \Rightarrow Xw = y$$

$$(QR)w=y\Rightarrow Rw =Q^{\top} y$$

$$\text{QR-decomposition}+\text{Backward-Substitution}$$



In [27]:
def ols_via_qr(X, y):
    """
    Compute the OLS estimator beta_hat for y = X beta + eps
    using QR decomposition (Gram-Schmidt) and back-substitution.

    X: (n x p) design matrix
    y: (n,) or (n,1) response vector

    Returns:
        beta_hat: (p,) vector of OLS coefficients
    """
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float).reshape(-1)  

    
    Q, R = qr_gram_schmidt(X) 
    
    b = Q.T @ y             

    beta_hat = back_substitution(R, b)

    return beta_hat


In [42]:
np.random.seed(42)   
n = 100 
p = 3 
X = np.random.randn(n, p)
beta_true = np.array([1.5, -2.0, 0.5])
sigma = 0.5
eps = sigma * np.random.randn(n)
y = X @ beta_true + eps
beta_hat = ols_via_qr(X, y)
print("True beta:     ", beta_true)
print("Estimated beta:", beta_hat)
beta_lstsq, *_ = np.linalg.lstsq(X, y, rcond=None)
print("np.linalg.lstsq beta:", beta_lstsq)

X = np.asarray(X, float)
y = np.asarray(y, float)

n, p = X.shape
XtX = X.T @ X
Xty = X.T @ y
I = np.eye(p)
lam = 1
beta_ridge = np.linalg.solve(XtX + lam * I, Xty)
beta_ridge

True beta:      [ 1.5 -2.   0.5]
Estimated beta: [ 1.46840505 -2.03466378  0.44907805]
np.linalg.lstsq beta: [ 1.46840505 -2.03466378  0.44907805]


array([ 1.44825316, -2.01574798,  0.44603553])

In [90]:
def ridge_regression_closed_form(X, y, lam):
    """
    Closed-form ridge regression:
        beta = (X^T X + lam * I)^(-1) X^T y

    Parameters
    ----------
    X : (n, p) array
    y : (n,) array
    lam : float, ridge penalty λ >= 0
    """
    X = np.asarray(X, float)
    y = np.asarray(y, float).ravel()

    n, p = X.shape
    XtX = X.T @ X
    Xty = X.T @ y
    I = np.eye(p)
    beta_ridge = np.linalg.solve(XtX + lam * I, Xty)
    return beta_ridge

In [91]:
np.random.seed(42)
n, p = 100, 5
X = np.random.randn(n, p)
beta_true = np.array([1.5, -2.0, 0.0, 0.5, 0.0])
y = X @ beta_true + 0.5 * np.random.randn(n)

lam = 1# L1 penalty
beta_hat = ridge_regression_closed_form(X, y, lam)
beta_hat

array([ 1.51209096, -1.92031595, -0.00426938,  0.56217399, -0.00372307])

In [85]:
import numpy as np
def soft_threshold(x, lam):
    if x > lam:
        return x - lam
    elif x < -lam:
        return x + lam
    else:
        return 0.0

def lasso_coordinate_descent(X, y, lam, max_iter=1000, tol=1e-6):
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float).ravel()
    n, p = X.shape

    X_mean = X.mean(axis=0)
    y_mean = y.mean()
    Xc = X - X_mean
    yc = y - y_mean

    beta = np.zeros(p)
    resid = yc.copy()

    for it in range(max_iter):
        max_change = 0.0

        for j in range(p):
            xj = Xc[:, j]
            resid += xj * beta[j]
            rho_j = (xj @ resid) / n
            z_j = (xj ** 2).sum() / n
            if z_j == 0:
                new_beta_j = 0.0
            else:
                new_beta_j = soft_threshold(rho_j, lam) / z_j

            resid -= xj * new_beta_j
            max_change = max(max_change, abs(new_beta_j - beta[j]))

            beta[j] = new_beta_j

        if max_change < tol:
            break
    beta0 = y_mean - X_mean @ beta
    return beta0, beta
if __name__ == "__main__":
    np.random.seed(42)
    n, p = 100, 5
    X = np.random.randn(n, p)
    beta_true = np.array([1.5, -2.0, 0.0, 0.5, 0.0])
    y = X @ beta_true + 0.5 * np.random.randn(n)

    lam = 1# L1 penalty
    beta0_hat, beta_hat = lasso_coordinate_descent(X, y, lam)

    print("True intercept ≈ 0.0 (because we simulated with no intercept)")
    print("Estimated intercept:", beta0_hat)
    print("True beta:     ", beta_true)
    print("Estimated beta:", beta_hat)

True intercept ≈ 0.0 (because we simulated with no intercept)
Estimated intercept: -0.11765437766603941
True beta:      [ 1.5 -2.   0.   0.5  0. ]
Estimated beta: [ 0.37727042 -1.04109465  0.          0.          0.        ]
